In [1]:
!pip install pandas prophet scikit-learn plotly yfinance

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from prophet.plot import plot_plotly, plot_components_plotly, plot_cross_validation_metric
from prophet import Prophet
from prophet.utilities import regressor_coefficients
from prophet.diagnostics import cross_validation

# create a dictionary to store the results
my_dict = {"Ticker": [], "Last Close": [], "Projected Price": [],
           "Projected Upside": [], "Predicted MSE": [], "Actual MSE": []}

# create an empty list for storing tickers
tickers = []

# function to calculate mean squared error (MSE)
def mse(predictions, actual_label="actual", pred_label="yhat"):
  se = ((predictions[actual_label]- predictions[pred_label]) ** 2)
  return(se.mean())

# function to fit the Prophet model
def fit_prophet(train):
  m = Prophet()
  for p in predictors:
    m.add_regressor(p)
  m.fit(train)
  return m

# path to the text file containing tickers
tickers_file = "C:\\Python\\tickers.txt"

# read the tickers from the text file
with open(tickers_file, 'r') as f:
    tickers = [line.strip() for line in f]

# loop through each ticker to perform forecasting
for ticker in tickers:
    # a flag for detecting exceptions
    exception = True
    errors = 0
    # try to download stock data
    while exception:
        exception = False
        try:
            
        #period = 'max' can be used instead of start and ending dates
          stock_data = \
          yf.download(ticker, start='2013-01-07', end='2022-01-03', interval='1d', auto_adjust=False, prepost=False, progress=False)
        except Exception as e:
            exception = True
            errors = errors + 1
            if errors > 10:
                print(ticker + " is most likely delisted. Attempts made: " + str(errors))
                exception = False
    try:       
      # drop columns with high missing values
      null_pct = stock_data.apply(pd.isnull).sum()/stock_data.shape[0]
      valid_columns = stock_data.columns[null_pct < 0.5]
      stock_data = stock_data[valid_columns].copy()
      # change column names to lower case
      stock_data.columns = stock_data.columns.str.lower()
      # convert index to UTC time
      stock_data.index = stock_data.index.tz_convert('UTC').tz_convert(None)
      # create target variable "y" as the high price of next day
      stock_data["y"] = stock_data.shift(-1)['high']
      # forward fill missing values
      stock_data = stock_data.ffill()
      # create a new column "ds" for the timestamp
      stock_data["ds"] = stock_data.index
      # get a list of predictors
      predictors = stock_data.columns[~stock_data.columns.isin(["y", "ds"])]
      #predictors

      split = round(len(stock_data) * 0.8)
      train = stock_data.iloc[:split]
      test = stock_data.iloc[split:]

      m = fit_prophet(train)
      predictions = m.predict(test)
      #predictions
      #plot_components_plotly(m, predictions)
      regressor_coefficients(m)
      predictions.index = test.index
      predictions["actual"] = test["y"]
      mse_calc = mse(predictions)
      #print("MSE for " + ticker + " is: " + str(mse_calc))
      m = fit_prophet(stock_data)
      cv = cross_validation(m, initial=f'{365 * 5} days', period='180 days', horizon='180 days', parallel="processes")
      actual_mse = mse(cv, actual_label="y")
      #cv[["y", "yhat"]][-365:].plot()
      m = fit_prophet(stock_data)
      m.predict(stock_data.iloc[-1:])
      m = Prophet()
      m.fit(stock_data)
      future = m.make_future_dataframe(periods=60)
      forecast = m.predict(future)
      #plot_plotly(m, forecast)
      #forecast
      forecasted_price = round(forecast.iloc[-1]['yhat'],2)
      #print("Predicted Price: " + str(forecasted_price))
      last_close = round(stock_data.iloc[-1]['close'],2)
      #print("Last Close: " + str(last_close))
      projected_upside = round((forecasted_price / last_close -1), 2)
      #print("Projected Upside: " + str(projected_upside) + "%")
      my_dict["Ticker"].append(ticker)
      my_dict["Last Close"].append(last_close)
      my_dict["Projected Price"].append(forecasted_price)
      my_dict["Projected Upside"].append(projected_upside)
      my_dict["Predicted MSE"].append(round(mse_calc,2))
      my_dict["Actual MSE"].append(round(actual_mse,2))
      print("Ticker: " + ticker + " | Last Close: " + str(last_close) + " | Projected Price: " + str(forecasted_price) + " | Projected Upside: " + str(projected_upside) + " | Predicted MSE: " + str(round(mse_calc,2)) + " | Actual MSE: " + str(round(actual_mse,2)))
    except Exception as e:
        pass
df = pd.DataFrame(my_dict)
print(df)
df.to_excel("prophet.xlsx")


1 Failed download:
- CEG: Data doesn't exist for startDate = 1357534800, endDate = 1641186000


04:08:32 - cmdstanpy - INFO - Chain [1] start processing
04:08:32 - cmdstanpy - INFO - Chain [1] done processing
04:08:32 - cmdstanpy - INFO - Chain [1] start processing
04:08:32 - cmdstanpy - INFO - Chain [1] done processing
04:08:32 - cmdstanpy - INFO - Chain [1] start processing
04:08:33 - cmdstanpy - INFO - Chain [1] done processing
04:08:33 - cmdstanpy - INFO - Chain [1] start processing
04:08:33 - cmdstanpy - INFO - Chain [1] done processing
04:08:34 - cmdstanpy - INFO - Chain [1] start processing
04:08:34 - cmdstanpy - INFO - Chain [1] done processing
04:08:35 - cmdstanpy - INFO - Chain [1] start processing
04:08:35 - cmdstanpy - INFO - Chain [1] done processing
04:08:37 - cmdstanpy - INFO - Chain [1] start processing
04:08:38 - cmdstanpy - INFO - Chain [1] done processing
04:08:38 - cmdstanpy - INFO - Chain [1] start processing
04:08:38 - cmdstanpy - INFO - Chain [1] done processing


Ticker: TSLA | Last Close: 352.26 | Projected Price: 374.81 | Projected Upside: 0.06 | Predicted MSE: 36.9 | Actual MSE: 18.52


04:08:40 - cmdstanpy - INFO - Chain [1] start processing
04:08:40 - cmdstanpy - INFO - Chain [1] done processing
04:08:41 - cmdstanpy - INFO - Chain [1] start processing
04:08:41 - cmdstanpy - INFO - Chain [1] done processing
04:08:42 - cmdstanpy - INFO - Chain [1] start processing
04:08:43 - cmdstanpy - INFO - Chain [1] done processing
04:08:43 - cmdstanpy - INFO - Chain [1] start processing
04:08:43 - cmdstanpy - INFO - Chain [1] done processing


Ticker: PAYC | Last Close: 415.19 | Projected Price: 504.94 | Projected Upside: 0.22 | Predicted MSE: 51.03 | Actual MSE: 48.83


...

04:09:56 - cmdstanpy - INFO - Chain [1] start processing
04:09:56 - cmdstanpy - INFO - Chain [1] done processing
04:09:57 - cmdstanpy - INFO - Chain [1] start processing
04:09:57 - cmdstanpy - INFO - Chain [1] done processing
04:09:58 - cmdstanpy - INFO - Chain [1] start processing
04:09:59 - cmdstanpy - INFO - Chain [1] done processing
04:09:59 - cmdstanpy - INFO - Chain [1] start processing
04:09:59 - cmdstanpy - INFO - Chain [1] done processing


Ticker: TDG | Last Close: 636.28 | Projected Price: 695.62 | Projected Upside: 0.09 | Predicted MSE: 147.07 | Actual MSE: 96.07


04:10:00 - cmdstanpy - INFO - Chain [1] start processing
04:10:01 - cmdstanpy - INFO - Chain [1] done processing
04:10:01 - cmdstanpy - INFO - Chain [1] start processing
04:10:02 - cmdstanpy - INFO - Chain [1] done processing
04:10:03 - cmdstanpy - INFO - Chain [1] start processing
04:10:04 - cmdstanpy - INFO - Chain [1] done processing
04:10:04 - cmdstanpy - INFO - Chain [1] start processing
04:10:05 - cmdstanpy - INFO - Chain [1] done processing


Ticker: HD | Last Close: 415.01 | Projected Price: 384.93 | Projected Upside: -0.07 | Predicted MSE: 12.49 | Actual MSE: 9.51


04:10:07 - cmdstanpy - INFO - Chain [1] start processing
04:10:07 - cmdstanpy - INFO - Chain [1] done processing
04:10:08 - cmdstanpy - INFO - Chain [1] start processing
04:10:08 - cmdstanpy - INFO - Chain [1] done processing
04:10:09 - cmdstanpy - INFO - Chain [1] start processing
04:10:10 - cmdstanpy - INFO - Chain [1] done processing
